In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import folium
import branca.colormap as cm

## Load and Inspect Earthquake Data

We load the cleaned earthquake dataset, then create latitude and longitude bins to group earthquake events into 5°x5° regions for risk mapping.

In [2]:
# Load earthquake data
df = pd.read_csv('../data/cleaned_earthquake_data.csv')

# Create latitude and longitude bins (5-degree intervals)
df['lat_bin'] = pd.cut(df['latitude'], bins=np.arange(-90, 91, 5))
df['lon_bin'] = pd.cut(df['longitude'], bins=np.arange(-180, 181, 5))


## Compute Regional Statistics and Risk Score

We group the data by latitude and longitude bins and calculate:
- Average magnitude (`mag_mean`)
- Event count per bin (`event_count`)
- Average depth (`depth_mean`)

Then we calculate a **custom risk score** using the formula:

> risk_score = (2 × mag_mean + event_count) / (depth_mean + 1)

This emphasizes higher magnitude and more frequent but shallow earthquakes.


In [3]:
# Group by lat/lon bins and compute stats
region_stats = df.groupby(['lat_bin', 'lon_bin'], observed=True).agg({
    'mag': ['mean', 'count'],
    'depth': 'mean'
}).reset_index()

# Rename columns
region_stats.columns = ['lat_bin', 'lon_bin', 'mag_mean', 'event_count', 'depth_mean']

# Risk score formula
region_stats['risk_score'] = (region_stats['mag_mean'] * 2 + region_stats['event_count']) / (region_stats['depth_mean'] + 1)


## Normalize Risk Scores and Map to Colors

We normalize the risk scores between 0 and 1 and use a `YlOrRd` (Yellow-Orange-Red) colormap to assign a color based on risk intensity.


In [4]:
# Normalize the risk scores between 0 and 1
min_risk = region_stats['risk_score'].min()
max_risk = region_stats['risk_score'].max()
region_stats['norm_risk'] = (region_stats['risk_score'] - min_risk) / (max_risk - min_risk)

# Create color map
colormap = cm.linear.YlOrRd_09.scale(0, 1)

# Apply color based on normalized risk
region_stats['color'] = region_stats['norm_risk'].apply(colormap)


## Visualize Regional Earthquake Risk on Interactive Map

Each rectangular region (5°x5°) is shown in a color representing risk. Redder zones indicate areas with higher earthquake risk.


In [5]:
# Initialize base map centered at global mean coordinates
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=2)

# Plot each region as a colored rectangle
for _, row in region_stats.iterrows():
    lat_range = row['lat_bin']
    lon_range = row['lon_bin']

    # Skip any NaN bins
    if pd.isnull(lat_range) or pd.isnull(lon_range):
        continue

    bounds = [
        [lat_range.left, lon_range.left],
        [lat_range.right, lon_range.right]
    ]

    folium.Rectangle(
        bounds=bounds,
        color=row['color'],
        fill=True,
        fill_opacity=0.7,
        weight=0.5,
        popup=(
            f"Mag Mean: {row['mag_mean']:.2f}<br>"
            f"Depth Mean: {row['depth_mean']:.2f} km<br>"
            f"Event Count: {row['event_count']}<br>"
            f"Risk Score: {row['risk_score']:.2f}"
        )
    ).add_to(m)

# Add the color legend
colormap.caption = "Earthquake Risk Score"
colormap.add_to(m)

# Display the map
m


In [6]:
m.save('earthquake_risk_map.html')

## 🌍 Interactive Earthquake Risk Map

[🔗 View Interactive Map](https://github.com/<your-username>/earthquake-risk-mapping/blob/main/output/earthquake_risk_map.html)

![🖼️ Map Screenshot](https://github.com/<your-username>/earthquake-risk-mapping/raw/main/output/earthquake_risk_map.png)


## 🧾 Conclusion

This interactive risk map provides a valuable overview of earthquake-prone regions based on a combined analysis of earthquake frequency, magnitude, and depth. By dividing the globe into 5°×5° bins and calculating a custom risk score, we can easily identify areas that are potentially more hazardous.

- **Regions with higher frequency and stronger, shallow earthquakes** are highlighted in **orange to red**, signaling elevated risk.
- **Less active or deeper earthquake zones** appear in **yellow**, indicating lower relative risk.

This tool can support geoscientific research, emergency preparedness, infrastructure planning, and public safety initiatives. Future improvements may include integrating seismic hazard models, population exposure data, or real-time monitoring to enhance predictive capability.
